# Halfar Dome Model

Replicating Luke Morris and Jadon Clugston's notebooks
* https://www.cise.ufl.edu/~luke.morris/cism.html
* https://github.com/JuliaComputing/ASKEMDemos/blob/main/Glacial%20Flow/GlacialFlowNotebook.jl

In [1]:
using Catlab
using Catlab.Graphics
using CombinatorialSpaces
using Decapodes

using MLStyle
using MultiScaleArrays
using LinearAlgebra
using OrdinaryDiffEq
using SparseArrays
using Statistics
# using BenchmarkTools

import Pkg
Pkg.add("JLD2")
# Pkg.add("GLMakie")
Pkg.add("CairoMakie")
Pkg.add("GeometryBasics")
Pkg.add("FileIO")
Pkg.add("MeshIO")
Pkg.add("DataFrames")
using GeometryBasics: Point2, Point3
using JLD2
# using CairoMakie
using FileIO, MeshIO
using DataFrames

Pkg.add("CSV")
import CSV

Point2D = Point2{Float64}
Point3D = Point3{Float64}

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.9/Project.toml`
  [5789e2e9] + FileIO v1.16.1
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.9/Project.toml`
  [7269a6da] + MeshIO v0.4.10
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
   Installed InlineStrings ── v1.4.0
   Installed PooledArrays ─── v1.4.3
   Installed SentinelArrays ─

Point3{Float64} (alias for GeometryBasics.Point{3, Float64})

In [2]:
Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [3]:
Pkg.add("SyntacticModels")
using SyntacticModels.AMR
using SyntacticModels.ASKEMDecapodes
using SyntacticModels.ASKEMUWDs
using SyntacticModels.Composites

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
[ Info: Precompiling SyntacticModels [22bb929c-8bcf-4852-b455-eb3e1675e09c]
┌ Warning: Module Decapodes with build ID fafbfcfd-b9a5-1825-0087-064bde04b5f4 is missing from the cache.
│ This may mean Decapodes [679ab3ea-c928-4fe6-8d59-fd451142d391] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1793
[ Info: Skipping precompilation since __precompile__(false). Importing SyntacticModels [22bb929c-8bcf-4852-b455-eb3e1675e09c].


In [4]:
Pkg.add("NetCDF")
import NetCDF

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


## Specify Models

Models can be defined in two ways:
1. SummationDecapode
2. DecaExpr

(1) stores the model as a graph while (2) stores it as an abstract syntax tree.
The latter is preferred for reconstructing the equations of the model.

In [5]:
# Halfar Model

# as a SummationDecapode
halfar_model_sm = @decapode begin
  h::Form0
  Γ::Form1
  n::Constant

  ḣ == ∂ₜ(h)
  ḣ == ∘(⋆, d, ⋆)(Γ * d(h) * avg₀₁(mag(♯(d(h)))^(n-1)) * avg₀₁(h^(n+2)))
end

# Serialize as AMR JSON
open("halfar_model_sm.json", "w") do io 
    d = Dict("header" => AMR.Header("halfar_model", "modelreps.io/SummationDecapode", "Halfar Model", "SummationDecapode", "v1.0"), 
        "model" => generate_json_acset(halfar_model_sm))
    JSON3.pretty(io, d, JSON3.AlignmentContext(indent = 2))
end

In [212]:
# as a DecaExpr
halfar_model_de = ASKEMDecaExpr(
    AMR.Header("halfar_model", "modelreps.io/DecaExpr", "Halfar Model", "DecaExpr", "v1.0"), 
    Decapodes.parse_decapode(quote
      h::Form0
      Γ::Form1
      n::Constant

      ḣ == ∂ₜ(h)
      ḣ == ∘(⋆, d, ⋆)(Γ * d(h) * avg₀₁(mag(♯(d(h)))^(n-1)) * avg₀₁(h^(n+2)))
    end)
)

# Serialize as AMR JSON
open("halfar_model_de.json", "w") do io 
    JSON3.pretty(io, halfar_model_de, JSON3.AlignmentContext(indent = 2))
end

In [7]:
# Glen's Law Model

# as a SummationDecapode
glen_model_sm = @decapode begin
    Γ::Form1
    A::Constant
    ρ::Constant
    g::Constant
    n::Constant
  
    Γ == (2/(n+2)) * A * (ρ * g)^n
end

# Serialize as AMR JSON
open("glen_model_sm.json", "w") do io 
    d = Dict("header" => AMR.Header("glen_model", "modelreps.io/SummationDecapode", "Glen's Law Model", "SummationDecapode", "v1.0"), 
        "model" => generate_json_acset(glen_model_sm))
    JSON3.pretty(io, d, JSON3.AlignmentContext(indent = 2))
end

# as a DecaExpr
glen_model_de = ASKEMDecaExpr(
    AMR.Header("glen_model", "modelreps.io/DecaExpr", "Glen Model", "DecaExpr", "v1.0"), 
    Decapodes.parse_decapode(quote
        Γ::Form1
        A::Constant
        ρ::Constant
        g::Constant
        n::Constant

        Γ == (2/(n+2)) * A * (ρ * g)^n
    end)
)

# Save to AMR JSON
open("glen_model_de.json", "w") do io 
  JSON3.pretty(io, glen_model_de, JSON3.AlignmentContext(indent = 2))
end

## Compose Models

Several smaller "component" models can be composed together into a "composite" model:
1. Specify a composition (undirected wiring) diagram that connects common variables and parameters together
2. Plug in the component models
3. Optionally flatten the composite model to remove the composition structure

In [10]:
# Specify the composition diagram
dome_model_uwd = @relation () begin
    dynamics(Γ, n)
    stress(Γ, n)
end

# Serialize as JSON
write_json_acset(dome_model_uwd, "dome_model_uwd.json")

298

In [11]:
# Plug in the component models

# using SummationDecapode
dome_model_sm = apex(oapply(dome_model_uwd, [
        Open(halfar_model_sm, [:Γ, :n]), 
        Open(glen_model_sm, [:Γ, :n])
]))

# Save to AMR JSON
open("dome_model_sm.json", "w") do io 
    d = Dict("header" => AMR.Header("dome_model", "modelreps.io/SummationDecapode", "Dome model as flattened composite of Halfar and Glen's law", "SummationDecapode", "v1.0"), 
        "model" => generate_json_acset(dome_model_sm))
    JSON3.pretty(io, d, JSON3.AlignmentContext(indent = 2))
end

In [12]:
# Specify the composition diagram again using UWDExpr
dome_model_uwde = ASKEMUWDs.UWDExpr(
    [Typed(:Γ, :Form1), Typed(:n, :Constant)], 
    [Statement(:dynamics, [Typed(:Γ, :Form1), Typed(:n, :Constant)]), Statement(:stress, [Typed(:Γ, :Form1), Typed(:n, :Constant)])]
)

# Serialize
open("dome_model_uwde.json", "w") do io 
  JSON3.pretty(io, dome_model_uwde, JSON3.AlignmentContext(indent = 2))
end

In [13]:
# Repeat using DecaExpr, UWDExpr, CompositeModelExpr
dome_model_de_comp = CompositeModelExpr(
    AMR.Header("dome_model", "modelreps.io/Composite", "Dome model as composite of Halfar and Glen's law", "CompositeModelExpr", "v0.0"),
    dome_model_uwde,
    [
        OpenModel(halfar_model_de, [:Γ, :n]), 
        OpenModel(glen_model_de, [:Γ, :n])
    ]
)

# Serialize directly as a CompositeModelExpr
open("dome_model_de_comp.json", "w") do io 
    JSON3.pretty(io, dome_model_de_comp, JSON3.AlignmentContext(indent = 2))
end

# Flatten and serialize as a SummationDecapode
open("dome_model_de_flat_sm.json", "w") do io 
    d = Dict("header" => AMR.Header("dome_model", "modelreps.io/SummationDecapode", "Dome model as flattened composite of Halfar and Glen's law", "SummationDecapode", "v1.0"), 
        "model" => generate_json_acset(OpenDecapode(dome_model_de_comp).model.model))
    JSON3.pretty(io, d, JSON3.AlignmentContext(indent = 2))
end

Serializing `CompositeModelExpr` directly appears to generate a non-compliant AMR JSON. Is that correct?

Also, there appears to be two ways to flatten a `CompositeModelExpr`: 
1. `apex(oapply(dome_model_de_comp))`
2. `OpenDecapode(dome_model_de_comp).model.model`

Which is the correct way?

Questions:
1. Is it possible to serialize non-flattened version of the `SummationDecapode` composite model (i.e. the cospan)? 
2. Is it also possible to flatten and then serialize the `CompositeModelExpr` composite model? Currently, `apex(dome_model_de)` throws an error.

## Specify Dimensionality

We need to specify the number of spatial dimensions for which the discrete exterior calculus operators in the model are interpreted.

In [14]:
# 1D
dome_model_1D = expand_operators(dome_model_sm)
infer_types!(dome_model_1D, op1_inf_rules_1D, op2_inf_rules_1D)
resolve_overloads!(dome_model_1D, op1_res_rules_1D, op2_res_rules_1D)

Var,type,name
1,Form0,dynamics_h
2,Form1,Γ
3,Constant,n
4,Form0,dynamics_ḣ
5,infer,dynamics_mult_1
6,Form1,dynamics_mult_2
7,Form1,dynamics_•1
8,infer,dynamics_•2
9,infer,dynamics_•3
10,infer,dynamics_•4


In [15]:
# 2D
dome_model_2D = expand_operators(dome_model_sm)
infer_types!(dome_model_2D)
resolve_overloads!(dome_model_2D)

Var,type,name
1,Form0,dynamics_h
2,Form1,Γ
3,Constant,n
4,Form0,dynamics_ḣ
5,infer,dynamics_mult_1
6,Form1,dynamics_mult_2
7,Form1,dynamics_•1
8,infer,dynamics_•2
9,infer,dynamics_•3
10,infer,dynamics_•4


## Configure Model 

Configure the model by defining a mesh. 
The mesh to be used to discretize the domain and the model needs to be defined:
* using helper functions
* uploading a shapefile
* using custom code

In [16]:
# Define a 1D mesh using a helper function

s_prime_1D = EmbeddedDeltaSet1D{Bool, Point2D}()
add_vertices!(s_prime_1D, 20, point = Point2D.(range(0, 10_000, length = 20), 0))
add_edges!(s_prime_1D, 1:nv(s_prime_1D) - 1, 2:nv(s_prime_1D))
orient!(s_prime_1D)
s_1D = EmbeddedDeltaDualComplex1D{Bool, Float64, Point2D}(s_prime_1D)
subdivide_duals!(s_1D, Circumcenter())

# Save both meshes as shapefile

LoadError: UndefVarError: `Mesh` not defined

In [17]:
# Define a 2D rectangular triangulated grid using a helper function

function triangulated_grid(max_x, max_y, dx, dy, point_type)

  s = EmbeddedDeltaSet2D{Bool, point_type}()

  # Place equally-spaced points in a max_x by max_y rectangle.
  coords = point_type == Point3D ? map(x -> point_type(x..., 0), Iterators.product(0:dx:max_x, 0:dy:max_y)) : map(x -> point_type(x...), Iterators.product(0:dx:max_x, 0:dy:max_y))
  # Perturb every other row right by half a dx.
  coords[:, 2:2:end] = map(coords[:, 2:2:end]) do row
    if point_type == Point3D
      row .+ [dx/2, 0, 0]
    else
      row .+ [dx/2, 0]
    end
  end
  # The perturbation moved the right-most points past max_x, so compress along x.
  map!(coords, coords) do coord
    if point_type == Point3D
      diagm([max_x/(max_x+dx/2), 1, 1]) * coord
    else
      diagm([max_x/(max_x+dx/2), 1]) * coord
    end
  end

  add_vertices!(s, length(coords), point = vec(coords))

  nx = length(0:dx:max_x)

  # Matrix that stores indices of points.
  idcs = reshape(eachindex(coords), size(coords))
  # Only grab vertices that will be the bottom-left corner of a subdivided square.
  idcs = idcs[begin:end-1, begin:end-1]
  
  # Subdivide every other row along the opposite diagonal.
  for i in idcs[:, begin+1:2:end]
    glue_sorted_triangle!(s, i, i+nx, i+nx+1)
    glue_sorted_triangle!(s, i, i+1, i+nx+1)
  end
  for i in idcs[:, begin:2:end]
    glue_sorted_triangle!(s, i, i+1, i+nx)
    glue_sorted_triangle!(s, i+1, i+nx, i+nx+1)
  end

  # Orient and return.
  s[:edge_orientation]=true
  orient!(s)
  s
end

In [18]:
# Define a 2D rectangle in 3D
s_prime_2D_rect = triangulated_grid(10_000, 10_000, 800, 800, Point3D)
s_2D_rect = EmbeddedDeltaDualComplex2D{Bool, Float64, Point3D}(s_prime_2D_rect)
subdivide_duals!(s_2D_rect, Barycenter())

# Save both meshes as shapefile

In [19]:
# Define a 2D sphere in 3D
s_prime_2D_sph = loadmesh(Icosphere(3, 10_000))
s_2D_sph = EmbeddedDeltaDualComplex2D{Bool, Float64, Point3D}(s_prime_2D_sph)
subdivide_duals!(s_2D_sph, Barycenter())

# Save both meshes as shapefile

In [20]:
# Define a 2D teapot in 3D
download("https://graphics.stanford.edu/courses/cs148-10-summer/as3/code/as3/teapot.obj", "teapot.obj")
s_prime_2D_tea = EmbeddedDeltaSet2D("teapot.obj")
s_2D_tea = EmbeddedDeltaDualComplex2D{Bool,Float64,Point3D}(s_prime_2D_tea)
subdivide_duals!(s_2D_tea, Circumcenter())

# Save both meshes as shapefile

# Parameterize Model

We need to specify values for:
* parameters (including constants)
* initial conditions
* boundary conditions (optional)

In [21]:
# Parameters
n = 3
ρ = 910
g = 9.8
A = 1e-16

1.0e-16

In [22]:
# Initial conditions

# 1D
h_init_1D = map(point(s_prime_1D)) do (x, _)
        ((7072 - ((x - 5000)^2)) / 9e3 + 2777) / 2777e-1
end

# 2D rectangular triangular grid
h_init_2D_rect = map(point(s_prime_2D_rect)) do (x, y)
  (7072 - ((x - 5000)^2 + (y - 5000)^2)^(1 / 2)) / 9e3 + 10
end

# 2D icosphere in 3D
h_init_2D_sph = map(point(s_prime_2D_sph)) do (x, y, z)
    (z * z) / (10_000 * 10_000)
end

# 2D teapot in 3D
h_init_2D_tea = map(point(s_prime_2D_tea)) do (x, y, z)
    abs(z) * 1.0
end

3644-element Vector{Float64}:
 0.0
 0.08100000023841858
 0.08100000023841858
 0.0
 0.0
 0.0
 0.08100000023841858
 0.08100000023841858
 0.08100000023841858
 0.08100000023841858
 0.14399999380111694
 0.14399999380111694
 0.14399999380111694
 ⋮
 0.06166800111532211
 0.06166800111532211
 0.0
 0.0
 0.05400000140070915
 0.05400000140070915
 0.0579960010945797
 0.0579960010945797
 0.0
 0.0
 0.0
 0.0

## Save initial conditions as NetCDF files

In [214]:
# 1D
f = joinpath("1D/h_init.nc")
isfile(f) && rm(f)
NetCDF.nccreate(
    f, "h_init", 
    "vertex_index", [i for i in eachindex(point(s_1D))], Dict("name" => "mesh-vertex index", "units" => "None"),
    atts = Dict("name" => "ice height at time = 0, 1D case", "units" => "m")
)
NetCDF.ncwrite(h_init_1D, f, "h_init")
NetCDF.ncinfo(f)

# 2D Rectangle
f = joinpath("2D_rect/h_init.nc")
isfile(f) && rm(f)
NetCDF.nccreate(
    f, "h_init", 
    "vertex_index", [i for i in eachindex(point(s_prime_2D_rect))], Dict("name" => "mesh-vertex index", "units" => "None"),
    atts = Dict("name" => "ice height at time = 0, 2D-rectangle in 2D space", "units" => "m")
)
NetCDF.ncwrite(h_init_2D_rect, f, "h_init")
NetCDF.ncinfo(f)

# 2D Icosphere in 3D
f = joinpath("2D_sph/h_init.nc")
isfile(f) && rm(f)
NetCDF.nccreate(
    f, "h_init", 
    "vertex_index", [i for i in eachindex(point(s_prime_2D_sph))], Dict("name" => "mesh-vertex index", "units" => "None"),
    atts = Dict("name" => "ice height at time = 0, 2D-icosphere in 3D sphere", "units" => "m")
)
NetCDF.ncwrite(h_init_2D_sph, f, "h_init")
NetCDF.ncinfo(f)

# 2D Teapot in 3D
f = joinpath("2D_tea/h_init.nc")
isfile(f) && rm(f)
NetCDF.nccreate(
    f, "h_init", 
    "vertex_index", [i for i in eachindex(point(s_prime_2D_tea))], Dict("name" => "mesh-vertex index", "units" => "None"),
    atts = Dict("name" => "ice height at time = 0, 2D-teapot in 3D sphere", "units" => "m")
)
NetCDF.ncwrite(h_init_2D_sph, f, "h_init")
NetCDF.ncinfo(f)


##### NetCDF File #####

/home/jovyan/work/Nelson/decapodes/dome_model/h_init_1D.nc

##### Dimensions #####

Name                                                Length                    
--------------------------------------------------------------------------------
vertex_index                                        20                        

##### Variables #####

Name                            Type            Dimensions                      
--------------------------------------------------------------------------------
vertex_index                    INT64           vertex_index                    
h_init                          DOUBLE          vertex_index                    

##### Attributes #####

Variable            Name                Value                                   
--------------------------------------------------------------------------------
vertex_index        units               None                                    
vertex_index        name           

In [23]:
extrema(h_init_2D_tea)

(0.0, 2.0)

In [24]:
# Boundary conditions
# None in this example

## Helper Functions

* `generate(...)`

In [25]:
# Implement DEC operators (♯, ♭, ∧, d, ⋆)
function generate(sd, my_symbol; hodge=GeometricHodge())
  op = @match my_symbol begin
    :♯ => x -> begin
      # This is an implementation of the "sharp" operator from the exterior
      # calculus, which takes co-vector fields to vector fields.
      # This could be up-streamed to the CombinatorialSpaces.jl library. (i.e.
      # this operation is not bespoke to this simulation.)
      e_vecs = map(edges(sd)) do e
        point(sd, sd[e, :∂v0]) - point(sd, sd[e, :∂v1])
      end
      neighbors = map(vertices(sd)) do v
        union(incident(sd, v, :∂v0), incident(sd, v, :∂v1))
      end
      n_vecs = map(neighbors) do es
        [e_vecs[e] for e in es]
      end
      map(neighbors, n_vecs) do es, nvs
        sum([nv*norm(nv)*x[e] for (e,nv) in zip(es,nvs)]) / sum(norm.(nvs))
      end
    end
    :mag => x -> begin
      norm.(x)
    end
    :avg₀₁ => x -> begin
      I = Vector{Int64}()
      J = Vector{Int64}()
      V = Vector{Float64}()
      for e in 1:ne(sd)
          append!(J, [sd[e,:∂v0],sd[e,:∂v1]])
          append!(I, [e,e])
          append!(V, [0.5, 0.5])
      end
      avg_mat = sparse(I,J,V)
      avg_mat * x
    end
    :^ => (x,y) -> x .^ y
    :* => (x,y) -> x .* y
    :show => x -> begin
      @show x
      x
    end
    x => error("Unmatched operator $my_symbol")
  end
  return (args...) -> op(args...)
end

generate (generic function with 1 method)

## Generate and Run Simulation

In [26]:
# Sim parameters
start_time = 0.0
end_time = 2e4

# Map constants to model parameters
constants_and_parameters = (
    n = n,
    stress_ρ = ρ,
    stress_g = g,
    stress_A = 1e-16
)

(n = 3, stress_ρ = 910, stress_g = 9.8, stress_A = 1.0e-16)

### 1D Case

In [215]:
# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_1D)], Float64[], [:dynamics_h])

# Generate simulation
sim = eval(gensim(dome_model_1D, dimension = 1))

# Implement DEC operators on the given mesh
fm = sim(s_1D , generate)

(::var"#f#123"{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, var"#50#63"{var"#47#60"}, var"#50#63"{var"#46#59"{EmbeddedDeltaDualComplex1D{Bool, Float64, Point2{Float64}}}}, var"#50#63"{var"#45#58"}, var"#50#63"{var"#38#51"{EmbeddedDeltaDualComplex1D{Bool, Float64, Point2{Float64}}}}, Diagonal{Float64, Vector{Float64}}, SparseMatrixCSC{Int64, Int64}, Diagonal{Float64, Vector{Float64}}, SparseMatrixCSC{Int64, Int64}}) (generic function with 1 method)

In [216]:
# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

[ Info: Precompiling Solver


true

In [217]:
# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
# @save "1D.jld2" soln
@info("Done")

[ Info: Solving


soln.retcode = SciMLBase.ReturnCode.Success


[ Info: Done


In [219]:
"ice height at time = 0, 1D case"
"ice height at time = 0, 2D-rectangle in 2D space"
"ice height at time = 0, 2D-icosphere in 3D sphere"
"ice height at time = 0, 2D-teapot in 3D sphere"

PhysicsState{VectorForm{Float64}, Float64}(VectorForm{Float64}[VectorForm{Float64}([2.8808066257871174e-5, 1.9950472470241813, 3.7683969705423332, 5.320077978620717, 6.650090271259331, 7.7584338484581785, 8.645108710217254, 9.310114856536563, 9.753452287416101, 9.975121002855868, 9.975121002855868, 9.753452287416101, 9.310114856536561, 8.645108710217254, 7.758433848458178, 6.650090271259331, 5.32007797862072, 3.7683969705423332, 1.9950472470241847, 2.8808066257871174e-5])], Float64[], [20], [:dynamics_h])

In [221]:
# Save solution
u = Array{Float64, 2}(undef, length(soln.t), length(soln.u[1]))
for i in eachindex(soln.t)
    for j in eachindex(soln.u[1])
        u[i, j] = soln.u[i][j]
    end
end

f = joinpath("1D/output.nc")
isfile(f) && rm(f)
NetCDF.nccreate(
    f, "h", 
    "t", soln.t, Dict("name" => "time", "units" => "day"), 
    "vertex_index", [i for i in eachindex(point(s_1D))], Dict("name" => "mesh-vertex index", "units" => "None"), 
    atts = Dict("name" => "ice height, 1D case", "units" => "m")
)
NetCDF.ncwrite(u, f, "h")
NetCDF.ncinfo(f)


##### NetCDF File #####

/home/jovyan/work/Nelson/decapodes/dome_model/1D.nc

##### Dimensions #####

Name                                                Length                    
--------------------------------------------------------------------------------
t                                                   56                        
vertex_index                                        20                        

##### Variables #####

Name                            Type            Dimensions                      
--------------------------------------------------------------------------------
t                               DOUBLE          t                               
h                               DOUBLE          t vertex_index                  
vertex_index                    INT64           vertex_index                    

##### Attributes #####

Variable            Name                Value                                   
--------------------------------------------

In [218]:
# df = DataFrame(soln)
# CSV.write("1D.csv", df)

### 2D Rectangular Triangulated Grid Case

In [222]:
# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_2D_rect)], Float64[], [:dynamics_h])

# Generate simulation
sim = eval(gensim(dome_model_2D, dimension = 2))

# Implement DEC operators on the given mesh
fm = sim(s_2D_rect , generate)

(::var"#f#124"{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, var"#50#63"{var"#47#60"}, var"#50#63"{var"#46#59"{EmbeddedDeltaDualComplex2D{Bool, Float64, Point3{Float64}}}}, var"#50#63"{var"#45#58"}, var"#50#63"{var"#38#51"{EmbeddedDeltaDualComplex2D{Bool, Float64, Point3{Float64}}}}, Diagonal{Float64, Vector{Float64}}, SparseMatrixCSC{Int64, Int64}, SparseMatrixCSC{Float64, Int64}, SparseMatrixCSC{Int64, Int64}}) (generic function with 1 method)

In [223]:
# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

[ Info: Precompiling Solver


true

In [224]:
# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
# @save "2D_rect.jld2" soln
@info("Done")

[ Info: Solving


soln.retcode = SciMLBase.ReturnCode.Success


[ Info: Done


In [202]:
point(s_2D_rect)

169-element Vector{Point3{Float64}}:
 [0.0, 0.0, 0.0]
 [769.2307692307693, 0.0, 0.0]
 [1538.4615384615386, 0.0, 0.0]
 [2307.6923076923076, 0.0, 0.0]
 [3076.923076923077, 0.0, 0.0]
 [3846.153846153846, 0.0, 0.0]
 [4615.384615384615, 0.0, 0.0]
 [5384.615384615385, 0.0, 0.0]
 [6153.846153846154, 0.0, 0.0]
 [6923.076923076923, 0.0, 0.0]
 [7692.307692307692, 0.0, 0.0]
 [8461.538461538461, 0.0, 0.0]
 [9230.76923076923, 0.0, 0.0]
 ⋮
 [769.2307692307693, 9600.0, 0.0]
 [1538.4615384615386, 9600.0, 0.0]
 [2307.6923076923076, 9600.0, 0.0]
 [3076.923076923077, 9600.0, 0.0]
 [3846.153846153846, 9600.0, 0.0]
 [4615.384615384615, 9600.0, 0.0]
 [5384.615384615385, 9600.0, 0.0]
 [6153.846153846154, 9600.0, 0.0]
 [6923.076923076923, 9600.0, 0.0]
 [7692.307692307692, 9600.0, 0.0]
 [8461.538461538461, 9600.0, 0.0]
 [9230.76923076923, 9600.0, 0.0]

In [230]:
# Save solution
u = Array{Float64, 2}(undef, length(soln.t), length(soln.u[1]))
for i in eachindex(soln.t)
    u[i, :] = [w for w in soln.u[i]]
end

f = joinpath("2D_rect/output.nc")
isfile(f) && rm(f)
NetCDF.nccreate(
    f, "h", 
    "t", soln.t, Dict("name" => "time", "units" => "day"), 
    "vertex_index", [i for i in eachindex(point(s_2D_rect))], Dict("name" => "mesh-vertex index", "units" => "m"),
    atts = Dict("name" => "ice height, 2D-rectangle in 2D space", "units" => "m")
)
NetCDF.ncwrite(u, f, "h")
NetCDF.ncinfo(f)


##### NetCDF File #####

/home/jovyan/work/Nelson/decapodes/dome_model/2D_rect.nc

##### Dimensions #####

Name                                                Length                    
--------------------------------------------------------------------------------
t                                                   6                         
vertex_index                                        169                       

##### Variables #####

Name                            Type            Dimensions                      
--------------------------------------------------------------------------------
t                               DOUBLE          t                               
h                               DOUBLE          t vertex_index                  
vertex_index                    INT64           vertex_index                    

##### Attributes #####

Variable            Name                Value                                   
---------------------------------------

In [193]:
# df = DataFrame(soln)
# CSV.write("2D_rect.csv", df)

### 2D Icosphere in 3D Case

In [227]:
# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_2D_sph)], Float64[], [:dynamics_h])

# Generate simulation
sim = eval(gensim(dome_model_2D, dimension = 2))

# Implement DEC operators on the given mesh
fm = sim(s_2D_sph, generate)

(::var"#f#127"{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, var"#50#63"{var"#47#60"}, var"#50#63"{var"#46#59"{EmbeddedDeltaDualComplex2D{Bool, Float64, Point3{Float64}}}}, var"#50#63"{var"#45#58"}, var"#50#63"{var"#38#51"{EmbeddedDeltaDualComplex2D{Bool, Float64, Point3{Float64}}}}, Diagonal{Float64, Vector{Float64}}, SparseMatrixCSC{Int64, Int64}, SparseMatrixCSC{Float64, Int64}, SparseMatrixCSC{Int64, Int64}}) (generic function with 1 method)

In [228]:
# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

[ Info: Precompiling Solver


true

In [229]:
# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
# @save "2D_sph.jld2" soln
@info("Done")

[ Info: Solving


soln.retcode = SciMLBase.ReturnCode.Success


[ Info: Done


In [234]:
# Save solution
u = Array{Float64, 2}(undef, length(soln.t), length(soln.u[1]))
for i in eachindex(soln.t)
    u[i, :] = [w for w in soln.u[i]]
end

f = joinpath("2D_sph/output.nc")
isfile(f) && rm(f)
NetCDF.nccreate(
    f, "h", 
    "t", soln.t, Dict("name" => "time", "units" => "day"), 
    "vertex_index", [i for i in eachindex(point(s_2D_sph))], Dict("name" => "mesh-vertex index", "units" => "m"),
    atts = Dict("name" => "ice height at time = 0, 2D-icosphere in 3D sphere", "units" => "m")
)
NetCDF.ncwrite(u, f, "h")
NetCDF.ncinfo(f)


##### NetCDF File #####

/home/jovyan/work/Nelson/decapodes/dome_model/2D_sph.nc

##### Dimensions #####

Name                                                Length                    
--------------------------------------------------------------------------------
t                                                   6                         
vertex_index                                        162                       

##### Variables #####

Name                            Type            Dimensions                      
--------------------------------------------------------------------------------
t                               DOUBLE          t                               
h                               DOUBLE          t vertex_index                  
vertex_index                    INT64           vertex_index                    

##### Attributes #####

Variable            Name                Value                                   
----------------------------------------

In [200]:
# Save solution
# df = DataFrame(soln)
# CSV.write("2D_sph.csv", df)

In [235]:
# 2D Teapot in 3D Case

# Map initial conditions to the state variable
u_init = construct(PhysicsState, [VectorForm(h_init_2D_tea)], Float64[], [:dynamics_h])

# Generate simulation
sim = eval(gensim(dome_model_2D, dimension = 2))

# Implement DEC operators on the given mesh
fm = sim(s_2D_tea , generate)

(::var"#f#128"{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, var"#50#63"{var"#47#60"}, var"#50#63"{var"#46#59"{EmbeddedDeltaDualComplex2D{Bool, Float64, Point3{Float64}}}}, var"#50#63"{var"#45#58"}, var"#50#63"{var"#38#51"{EmbeddedDeltaDualComplex2D{Bool, Float64, Point3{Float64}}}}, Diagonal{Float64, Vector{Float64}}, SparseMatrixCSC{Int64, Int64}, SparseMatrixCSC{Float64, Int64}, SparseMatrixCSC{Int64, Int64}}) (generic function with 1 method)

In [236]:
# Precompile
@info("Precompiling Solver")
prob = ODEProblem(fm, u_init, (start_time, start_time + 1e-8), constants_and_parameters)
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")

[ Info: Precompiling Solver


true

In [237]:
# Run
@info("Solving")
prob = ODEProblem(fm, u_init, (start_time, end_time), constants_and_parameters)
soln = solve(prob, Tsit5())
@show soln.retcode
# @save "2D_teapot.jld2" soln
@info("Done")

[ Info: Solving


soln.retcode = SciMLBase.ReturnCode.Success


[ Info: Done


In [247]:
extrema(findnode(soln(start_time), :dynamics_h))

(0.0, 2.0)

In [246]:
extrema(findnode(soln(end_time), :dynamics_h))

(-2.908776833042548e-6, 1.9980160202000756)

In [245]:
# Save solution
u = Array{Float64, 2}(undef, length(soln.t), length(soln.u[1]))
for i in eachindex(soln.t)
    u[i, :] = [w for w in soln.u[i]]
end

f = joinpath("2D_tea/output.nc")
isfile(f) && rm(f)
NetCDF.nccreate(
    f, "h", 
    "t", soln.t, Dict("name" => "time", "units" => "day"), 
    "vertex_index", [i for i in eachindex(point(s_2D_tea))], Dict("name" => "mesh-vertex index", "units" => "m"),
    atts = Dict("name" => "ice height at time = 0, 2D-teapot in 3D sphere", "units" => "m")
)
NetCDF.ncwrite(u, f, "h")
NetCDF.ncinfo(f)


##### NetCDF File #####

/home/jovyan/work/Nelson/decapodes/dome_model/2D_tea.nc

##### Dimensions #####

Name                                                Length                    
--------------------------------------------------------------------------------
t                                                   8                         
vertex_index                                        3644                      

##### Variables #####

Name                            Type            Dimensions                      
--------------------------------------------------------------------------------
t                               DOUBLE          t                               
h                               DOUBLE          t vertex_index                  
vertex_index                    INT64           vertex_index                    

##### Attributes #####

Variable            Name                Value                                   
----------------------------------------